# 🎯 Chapter 6 练习 3: 用户定向干预 - 因果推断的终极应用

## 把所有技术串起来!

恭喜你来到最后一个练习！这里我们将综合运用前面学到的所有技术：

- **CATE 估计**: T-Learner, X-Learner
- **用户分群**: 识别高价值用户
- **策略优化**: 成本-收益权衡

---

## 🎯 学习目标

1. 掌握 T-Learner 和 X-Learner 方法
2. 学习最优干预策略 (Policy Learning)
3. 理解成本-收益权衡
4. 进行敏感性分析

## 🌟 场景: 网约车司机激励

你是一家网约车平台的运营经理，需要决定：

> **给哪些司机发奖励？每个奖励花费 ¥100，希望司机多在线，每多在线 1 小时平台能赚 ¥30**

### 关键洞察

不同司机对激励的反应不同：

| 司机类型 | 激励效果 | 原因 |
|----------|----------|------|
| 兼职司机 | +2 小时 | 对激励敏感 |
| 全职司机 | +0.5 小时 | 已经很活跃，边际效应小 |
| 新手司机 | +2.5 小时 | 需要激励来建立习惯 |
| 老司机 | +1 小时 | 已经有稳定习惯 |

**关键**: 要找到那些「激励效果好」的司机，而不是「本来就活跃」的司机！

## 📐 核心公式

### 最优决策规则

$$\text{干预} \Leftrightarrow \text{CATE}(x) \times \text{Value} > \text{Cost}$$

即：只有当预期收益超过成本时才干预。

### X-Learner 公式

**Stage 1**: 训练两个结果模型
$$\hat{\mu}_0(x) = \mathbb{E}[Y|X=x, T=0]$$
$$\hat{\mu}_1(x) = \mathbb{E}[Y|X=x, T=1]$$

**Stage 2**: 计算伪处理效应
$$D_i^1 = Y_i - \hat{\mu}_0(X_i) \quad \text{(处理组)}$$
$$D_i^0 = \hat{\mu}_1(X_i) - Y_i \quad \text{(控制组)}$$

**Stage 3**: 加权组合
$$\hat{\tau}(x) = g(x) \cdot \hat{\tau}_0(x) + (1-g(x)) \cdot \hat{\tau}_1(x)$$

其中 $g(x)$ 是倾向得分。

In [ ]:
# 环境设置
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Dict
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression

# 设置随机种子
np.random.seed(42)

# 绘图设置
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

print("环境配置完成! 🎯")

## 练习 3.1: 生成司机激励数据

In [ ]:
def generate_driver_data(
    n_samples: int = 2000,
    seed: int = 42
) -> pd.DataFrame:
    """
    生成网约车司机激励数据
    
    场景: 平台给司机发放奖励，激励其增加在线时长
    """
    np.random.seed(seed)
    
    # TODO: 生成司机特征
    # rating: 评分 4.0-5.0
    rating = None  # 你的代码: np.random.beta(8, 1, n_samples) * 1.0 + 4.0
    
    # order_history: 历史完单数
    order_history = None  # 你的代码: np.random.poisson(200, n_samples)
    
    # is_fulltime: 是否全职 (30% 概率)
    is_fulltime = None  # 你的代码: np.random.binomial(1, 0.3, n_samples)
    
    # TODO: 随机分配处理
    T = None  # 你的代码
    
    # 生成在线时长
    online_hours = []
    for i in range(n_samples):
        if rating is not None and order_history is not None and is_fulltime is not None and T is not None:
            # 基线时长
            if is_fulltime[i] == 1:
                base = 6 + np.random.randn() * 1.5
            else:
                base = 3 + np.random.randn() * 1.0
            
            # 激励效应 (异质性!)
            if T[i] == 1:
                # 兼职效应更大
                fulltime_effect = 0.5 if is_fulltime[i] == 1 else 2.0
                # 低单量效应更大
                order_effect = 2.5 if order_history[i] < 150 else 1.0
                # 综合效应
                effect = (fulltime_effect + order_effect) / 2
            else:
                effect = 0
            
            online_hours.append(max(0, base + effect))
        else:
            online_hours.append(0)
    
    return pd.DataFrame({
        'rating': rating,
        'order_history': order_history,
        'is_fulltime': is_fulltime,
        'T': T,
        'online_hours': online_hours
    })

In [ ]:
# 生成数据
df = generate_driver_data(n_samples=2000)

if df is not None and df['rating'].iloc[0] is not None:
    print("司机数据生成成功! 🚗")
    print(f"\n样本量: {len(df)}")
    print(f"全职司机占比: {df['is_fulltime'].mean():.2%}")
    print(f"平均在线时长: {df['online_hours'].mean():.1f} 小时")
    print(f"激励组占比: {df['T'].mean():.2%}")
    
    # 按组查看
    print(f"\n=== 各组平均在线时长 ===")
    for is_ft in [0, 1]:
        for t in [0, 1]:
            mask = (df['is_fulltime']==is_ft) & (df['T']==t)
            ft_str = '全职' if is_ft == 1 else '兼职'
            t_str = '激励' if t == 1 else '无激励'
            print(f"  {ft_str} + {t_str}: {df.loc[mask, 'online_hours'].mean():.2f} 小时")
else:
    print("[TODO] 请完成 generate_driver_data 函数")

## 练习 3.2: T-Learner CATE 估计

In [ ]:
class TLearner:
    """
    T-Learner: 分别训练处理组和控制组的模型
    """
    
    def __init__(self):
        self.model_control = GradientBoostingRegressor(
            n_estimators=50, max_depth=4, random_state=42
        )
        self.model_treatment = GradientBoostingRegressor(
            n_estimators=50, max_depth=4, random_state=43
        )
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """训练 T-Learner"""
        # TODO: 分离控制组和处理组
        mask_control = None  # 你的代码: T == 0
        mask_treatment = None  # 你的代码: T == 1
        
        # TODO: 训练两个模型
        # self.model_control.fit(X[mask_control], Y[mask_control])
        # self.model_treatment.fit(X[mask_treatment], Y[mask_treatment])
        
        # 你的代码
        
        return self
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        """预测 CATE"""
        # TODO: 分别预测
        mu1 = None  # 你的代码
        mu0 = None  # 你的代码
        
        # TODO: CATE = mu1 - mu0
        cate = None  # 你的代码
        
        return cate

In [ ]:
# 训练 T-Learner
if df is not None and df['rating'].iloc[0] is not None:
    X = df[['rating', 'order_history', 'is_fulltime']].values
    T = df['T'].values
    Y = df['online_hours'].values
    
    try:
        t_learner = TLearner()
        t_learner.fit(X, T, Y)
        cate_t = t_learner.predict_cate(X)
        
        if cate_t is not None:
            print("T-Learner 训练成功! 🎉")
            print(f"\nCATE 统计:")
            print(f"  范围: [{cate_t.min():.2f}, {cate_t.max():.2f}] 小时")
            print(f"  均值: {cate_t.mean():.2f} 小时")
            print(f"  标准差: {cate_t.std():.2f} 小时")
            
            # 按司机类型查看
            print(f"\n各类型司机的预测 CATE:")
            print(f"  全职: {cate_t[df['is_fulltime']==1].mean():.2f} 小时")
            print(f"  兼职: {cate_t[df['is_fulltime']==0].mean():.2f} 小时")
        else:
            print("[TODO] 请完成 TLearner.predict_cate 方法")
    except Exception as e:
        print(f"[TODO] TLearner 实现有误: {e}")

## 练习 3.3: X-Learner CATE 估计

X-Learner 是更高级的 Meta-Learner，通常比 T-Learner 更准确。

In [ ]:
class XLearner:
    """
    X-Learner: 额外学习伪处理效应
    """
    
    def __init__(self):
        self.model_control = GradientBoostingRegressor(n_estimators=50, max_depth=4, random_state=42)
        self.model_treatment = GradientBoostingRegressor(n_estimators=50, max_depth=4, random_state=43)
        self.tau_control = GradientBoostingRegressor(n_estimators=30, max_depth=3, random_state=44)
        self.tau_treatment = GradientBoostingRegressor(n_estimators=30, max_depth=3, random_state=45)
        self.propensity_model = LogisticRegression(random_state=46)
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """训练 X-Learner (三阶段)"""
        mask_control = (T == 0)
        mask_treatment = (T == 1)
        
        # TODO: Stage 1 - 训练 mu_0 和 mu_1
        # 你的代码
        
        # TODO: Stage 2 - 计算伪处理效应
        # D_treatment = Y_treatment - mu_0(X_treatment)  # 处理组: 实际 - 反事实
        # D_control = mu_1(X_control) - Y_control        # 控制组: 反事实 - 实际
        # 你的代码
        
        # TODO: Stage 3 - 估计倾向得分
        # 你的代码
        
        return self
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        """预测 CATE (使用倾向得分加权)"""
        # TODO: 预测两个 tau
        tau0 = None  # 你的代码
        tau1 = None  # 你的代码
        
        # TODO: 估计倾向得分
        propensity = None  # 你的代码
        
        # TODO: 加权组合
        # cate = propensity * tau0 + (1 - propensity) * tau1
        cate = None  # 你的代码
        
        return cate

In [ ]:
# 训练 X-Learner
if df is not None and df['rating'].iloc[0] is not None:
    try:
        x_learner = XLearner()
        x_learner.fit(X, T, Y)
        cate_x = x_learner.predict_cate(X)
        
        if cate_x is not None:
            print("X-Learner 训练成功! 🎉")
            print(f"\nCATE 统计:")
            print(f"  范围: [{cate_x.min():.2f}, {cate_x.max():.2f}] 小时")
            print(f"  均值: {cate_x.mean():.2f} 小时")
            
            if cate_t is not None:
                corr = np.corrcoef(cate_t, cate_x)[0, 1]
                print(f"\n与 T-Learner 的相关性: {corr:.3f}")
        else:
            print("[TODO] 请完成 XLearner.predict_cate 方法")
    except Exception as e:
        print(f"[TODO] XLearner 实现有误: {e}")
        # 使用 T-Learner 的结果继续
        if 'cate_t' in dir() and cate_t is not None:
            cate_x = cate_t
            print("使用 T-Learner 的结果继续...")

## 练习 3.4: 最优策略学习

现在我们来学习最优的激励策略！

In [ ]:
def learn_optimal_policy(
    cate: np.ndarray,
    cost_per_treatment: float = 100,
    value_per_hour: float = 30
) -> Tuple[np.ndarray, Dict]:
    """
    学习最优干预策略
    
    决策规则: 当 CATE * value > cost 时进行干预
    
    Args:
        cate: 估计的 CATE (在线时长增量)
        cost_per_treatment: 每次激励的成本 (元)
        value_per_hour: 每小时的价值 (元)
    """
    # TODO: 计算阈值
    # 多少小时增量才划算? cost / value
    threshold = None  # 你的代码
    
    # TODO: 最优策略
    optimal_policy = None  # 你的代码: (cate > threshold).astype(int)
    
    if optimal_policy is None:
        return None, {}
    
    # TODO: 计算指标
    n_treated = optimal_policy.sum()
    expected_hours = cate[optimal_policy == 1].sum() if n_treated > 0 else 0
    total_cost = None  # 你的代码: n_treated * cost_per_treatment
    total_value = None  # 你的代码: expected_hours * value_per_hour
    net_benefit = None  # 你的代码: total_value - total_cost
    roi = None  # 你的代码: net_benefit / total_cost if total_cost > 0 else 0
    
    metrics = {
        'n_treated': n_treated,
        'treatment_rate': n_treated / len(cate),
        'expected_hours': expected_hours,
        'total_cost': total_cost,
        'total_value': total_value,
        'net_benefit': net_benefit,
        'roi': roi,
        'threshold': threshold
    }
    
    return optimal_policy, metrics

In [ ]:
# 学习最优策略
if 'cate_x' in dir() and cate_x is not None:
    cate_to_use = cate_x if cate_x is not None else cate_t
    
    optimal_policy, metrics = learn_optimal_policy(
        cate_to_use, 
        cost_per_treatment=100, 
        value_per_hour=30
    )
    
    if optimal_policy is not None and metrics.get('total_cost') is not None:
        print("最优策略学习成功! 🎯")
        print("="*50)
        print(f"\n激励成本: ¥100/人")
        print(f"每小时价值: ¥30")
        print(f"阈值: {metrics['threshold']:.2f} 小时 (CATE 需要超过这个值才划算)")
        print(f"\n=== 最优策略结果 ===")
        print(f"干预人数: {metrics['n_treated']}")
        print(f"干预比例: {metrics['treatment_rate']:.2%}")
        print(f"预期时长增量: {metrics['expected_hours']:.0f} 小时")
        print(f"总成本: ¥{metrics['total_cost']:,.0f}")
        print(f"总价值: ¥{metrics['total_value']:,.0f}")
        print(f"净收益: ¥{metrics['net_benefit']:,.0f}")
        print(f"ROI: {metrics['roi']:.2f}")
    else:
        print("[TODO] 请完成 learn_optimal_policy 函数")

## 练习 3.5: 策略对比

In [ ]:
def compare_targeting_strategies(
    df: pd.DataFrame,
    cate: np.ndarray,
    cost: float = 100,
    value: float = 30
) -> pd.DataFrame:
    """
    对比不同干预策略
    """
    results = []
    
    # 策略 1 - No Treatment
    results.append({
        'strategy': 'No Treatment',
        'n_treated': 0,
        'cost': 0,
        'value': 0,
        'net_benefit': 0,
        'roi': 0
    })
    
    # TODO: 策略 2 - Treat All
    n_all = len(df)
    expected_hours_all = cate.sum()
    cost_all = n_all * cost
    value_all = expected_hours_all * value
    results.append({
        'strategy': 'Treat All',
        'n_treated': n_all,
        'cost': cost_all,
        'value': value_all,
        'net_benefit': value_all - cost_all,
        'roi': (value_all - cost_all) / cost_all if cost_all > 0 else 0
    })
    
    # TODO: 策略 3 - Treat Part-time Only (传统规则)
    parttime_mask = df['is_fulltime'] == 0
    n_parttime = parttime_mask.sum()
    expected_hours_parttime = cate[parttime_mask].sum()
    cost_parttime = n_parttime * cost
    value_parttime = expected_hours_parttime * value
    results.append({
        'strategy': 'Treat Part-time',
        'n_treated': n_parttime,
        'cost': cost_parttime,
        'value': value_parttime,
        'net_benefit': value_parttime - cost_parttime,
        'roi': (value_parttime - cost_parttime) / cost_parttime if cost_parttime > 0 else 0
    })
    
    # 策略 4 - Optimal Policy
    optimal_policy, optimal_metrics = learn_optimal_policy(cate, cost, value)
    if optimal_policy is not None and optimal_metrics.get('total_cost') is not None:
        results.append({
            'strategy': 'Optimal Policy',
            'n_treated': optimal_metrics['n_treated'],
            'cost': optimal_metrics['total_cost'],
            'value': optimal_metrics['total_value'],
            'net_benefit': optimal_metrics['net_benefit'],
            'roi': optimal_metrics['roi']
        })
    
    return pd.DataFrame(results)

In [ ]:
# 策略对比
if 'cate_to_use' in dir() and cate_to_use is not None:
    comparison = compare_targeting_strategies(df, cate_to_use, cost=100, value=30)
    
    if comparison is not None and len(comparison) > 0:
        print("策略对比:")
        print("="*70)
        display(comparison.round(0))
        
        # 找最佳策略
        best_idx = comparison['net_benefit'].idxmax()
        best_strategy = comparison.loc[best_idx, 'strategy']
        best_benefit = comparison.loc[best_idx, 'net_benefit']
        
        print(f"\n最佳策略: {best_strategy}")
        print(f"净收益: ¥{best_benefit:,.0f}")
        
        # 可视化
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        colors = ['#95a5a6', '#e74c3c', '#3498db', '#2ecc71']
        
        # 净收益对比
        bars = axes[0].bar(comparison['strategy'], comparison['net_benefit'], color=colors)
        axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        axes[0].set_ylabel('净收益 (¥)')
        axes[0].set_title('不同策略的净收益对比')
        axes[0].tick_params(axis='x', rotation=15)
        
        # ROI 对比
        bars = axes[1].bar(comparison['strategy'], comparison['roi'], color=colors)
        axes[1].axhline(y=0, color='red', linestyle='--')
        axes[1].set_ylabel('ROI')
        axes[1].set_title('不同策略的 ROI 对比')
        axes[1].tick_params(axis='x', rotation=15)
        
        plt.tight_layout()
        plt.show()

## 练习 3.6: 用户分层

In [ ]:
def segment_by_cate(
    df: pd.DataFrame,
    cate: np.ndarray
) -> pd.DataFrame:
    """
    根据 CATE 将用户分层
    """
    df = df.copy()
    df['cate'] = cate
    
    # TODO: 计算分位数
    p75 = np.percentile(cate, 75)
    p25 = np.percentile(cate, 25)
    
    # 分层
    segments = []
    for c in cate:
        if c >= p75:
            segments.append('High Impact')
        elif c >= p25:
            segments.append('Medium Impact')
        elif c > 0:
            segments.append('Low Impact')
        else:
            segments.append('Negative Impact')
    
    df['segment'] = segments
    return df

In [ ]:
# 用户分层
if 'cate_to_use' in dir() and cate_to_use is not None:
    df_segmented = segment_by_cate(df, cate_to_use)
    
    print("用户分层结果:")
    print("="*50)
    print(f"\n分层分布:")
    print(df_segmented['segment'].value_counts())
    
    print(f"\n各层统计:")
    for segment in ['High Impact', 'Medium Impact', 'Low Impact', 'Negative Impact']:
        mask = df_segmented['segment'] == segment
        if mask.sum() > 0:
            avg_cate = df_segmented.loc[mask, 'cate'].mean()
            avg_fulltime = df_segmented.loc[mask, 'is_fulltime'].mean()
            avg_orders = df_segmented.loc[mask, 'order_history'].mean()
            print(f"\n  {segment}:")
            print(f"    平均 CATE: {avg_cate:.2f} 小时")
            print(f"    全职比例: {avg_fulltime:.1%}")
            print(f"    平均历史订单: {avg_orders:.0f}")
    
    # 可视化
    fig, ax = plt.subplots(figsize=(10, 6))
    
    segment_order = ['High Impact', 'Medium Impact', 'Low Impact', 'Negative Impact']
    colors = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c']
    
    segment_cates = [df_segmented.loc[df_segmented['segment']==s, 'cate'].mean() 
                    for s in segment_order]
    
    bars = ax.bar(segment_order, segment_cates, color=colors)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_ylabel('平均 CATE (小时)')
    ax.set_title('各用户分层的平均激励效果')
    ax.tick_params(axis='x', rotation=15)
    
    plt.tight_layout()
    plt.show()

## 🤔 思考题

In [ ]:
# 思考题 1: T-Learner 和 X-Learner 的主要区别是什么？X-Learner 在什么情况下更优？

answer_1 = """
你的答案:



"""
print(answer_1)

In [ ]:
# 思考题 2: 最优策略 "CATE × value > cost" 的经济学直觉是什么？

answer_2 = """
你的答案:



"""
print(answer_2)

In [ ]:
# 思考题 3: 为什么要使用倾向得分来加权 X-Learner 的预测？

answer_3 = """
你的答案:



"""
print(answer_3)

In [ ]:
# 思考题 4: 在真实业务中，如何处理 CATE 估计的不确定性？

answer_4 = """
你的答案:



"""
print(answer_4)

In [ ]:
# 思考题 5: 如果激励效应会随时间衰减（激励疲劳），应该如何调整策略？

answer_5 = """
你的答案:



"""
print(answer_5)

## 📊 总结

### Meta-Learner 对比

| 方法 | 复杂度 | 优点 | 缺点 |
|------|--------|------|------|
| S-Learner | 低 | 简单 | 可能无法捕获异质性 |
| T-Learner | 中 | 直观 | 处理组和控制组不共享信息 |
| X-Learner | 高 | 更准确，适合不平衡数据 | 更复杂 |

### 最优策略公式

$$\pi^*(x) = \mathbb{1}[\hat{\tau}(x) \cdot \text{value} > \text{cost}]$$

### 业务应用框架

1. **数据准备**: 收集历史实验数据
2. **CATE 估计**: 使用 T/X-Learner
3. **用户分层**: 识别高价值用户
4. **策略优化**: 成本-收益权衡
5. **持续迭代**: A/B 测试验证

In [ ]:
print("🎯 恭喜完成用户定向干预练习!")
print("\n你已经学会了:")
print("  ✓ T-Learner 和 X-Learner CATE 估计")
print("  ✓ 最优干预策略学习")
print("  ✓ 成本-收益权衡")
print("  ✓ 用户分层分析")
print("\n🎉 恭喜完成所有章节!")
print("\n你现在已经掌握了:")
print("  ✓ 因果推断基础概念")
print("  ✓ 处理效应估计方法")
print("  ✓ Uplift 建模")
print("  ✓ 深度因果模型")
print("  ✓ 异质效应分析")
print("  ✓ 实际应用场景")
print("\n继续探索，继续学习! 🚀")